In [1]:
import os
import numpy as np
import pandas as pd
import nltk
import keras
import six
import matplotlib.pyplot as plt
import json
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize, RegexpTokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,AveragePooling1D,Flatten,concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import *

In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
stop_words = set(stopwords.words('spanish'))

[nltk_data] Downloading package punkt to /home/roberto/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/roberto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/roberto/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/roberto/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
#Abro el archivo en el que se encuentra el dataset de los problemas
with open('singleop.json', 'r') as f:
    datastore = json.load(f)

In [4]:
# Archivo que contiene un listado de nombres
nombres = pd.read_csv('nombres-2015.csv')
names = pd.read_csv('yob2019.txt', header=None)    

In [5]:
dataset2 = pd.read_csv('problemas_adicionales2.csv')

In [6]:
preguntas = []
respuestas = []
ecuaciones = []
alineacion = []

for item in datastore:
    preguntas.append(item['sQuestion'])
    respuestas.append(item['lSolutions'])
    ecuaciones.append(item['lEquations'])
    alineacion.append(item['lAlignments'])

In [7]:
# Necesito convertir el dataset en un problema de clasificacion para que la red neuronal pueda identificar
# si estoy tratando de resolver un problema de sumas, restas, multiplicaciones o divisiones.
# Esto va a crear una lista con el tipo de operacion y que va a ser el resultado a inferir.
operaciones = []
sumas =0
restas =0
multiplicaciones =0
divisiones = 0
otras = 0
#Clasifico las operaciones en 0 para sumas, 1 para restas, 2 para multiplicaciones, 3 para divisiones y 4 sino lo encuentro.
for operacion in ecuaciones:
    if (operacion[0].find('+')>=0):
        operaciones.append(0)
        sumas = sumas + 1
    elif (operacion[0].find('-') >= 0 ):
        operaciones.append(1)
        restas = restas + 1
    elif(operacion[0].find('*') >=0):
        operaciones.append(2)
        multiplicaciones = multiplicaciones + 1
    elif(operacion[0].find('/')):
        operaciones.append(3)
        divisiones = divisiones + 1
    else:
        operaciones.append(4)
        otras = otras + 1

print('Tengo ', sumas, ' sumas ', restas, ' restas, ', multiplicaciones, ' multiplicaciones, ', divisiones, ' divisiones y otras operaciones ', otras)

Tengo  159  sumas  162  restas,  117  multiplicaciones,  124  divisiones y otras operaciones  0


In [8]:
preguntas2 = dataset2['Preguntas'].tolist()
respuestas2 = dataset2['respuestas'].tolist()

In [9]:
preguntas3 = preguntas + preguntas2
respuestas3 = operaciones + respuestas2

In [10]:
#El listado de nombres lo voy a truncar a los 15K primeros, dado que el resto son nombres muy residuales.
nombres_ = nombres['nombre'][:15000]

In [11]:
names_= names[0]

In [12]:
nombres_ = nombres_.append(names_)

In [13]:
st_words = list(stop_words)

In [14]:
nom = nombres['nombre'].values.tolist()

In [15]:
nomb =  nom + st_words

In [16]:
nomb[-1]

'fuimos'

In [17]:
# El vector preguntas_sin, consiste en las preguntas a las que voy a eliminar todos los nombres propios que no
# anaden ningun valor al conjunto de preguntas. No quiero que esos nombres se procesen y por tanto los elimino.
def eliminar_palabras(dataset, stopw):
    preguntas_sin = []
    for palabras in dataset:
        frases = [word for word in palabras.split(' ') if word not in stopw]
        frases = " ".join(frases)
        preguntas_sin.append(frases)
    return preguntas_sin

In [18]:
preguntas_sin = eliminar_palabras(preguntas3, nomb)

In [19]:
from sklearn.utils import shuffle

preguntas3, respuestas3 = shuffle(preguntas_sin,respuestas3)

In [20]:
preguntas_w = []
tokenizer = RegexpTokenizer(r'\w+')

for i,pregunta in enumerate(preguntas_sin):
    palabras = tokenizer.tokenize(pregunta)
    for palabra in palabras:
        preguntas_w.append(palabra)

In [21]:
palabras_unicas = set(preguntas_w)


In [22]:
p_unicas = list(palabras_unicas)

vocabulario = {p:i for i, p in enumerate(p_unicas)}
    
    

In [23]:
import gensim
import gensim.downloader as api
from gensim.models import KeyedVectors

In [24]:
glove = gensim.models.KeyedVectors.load_word2vec_format('SBW-vectors-300-min5.txt')

In [25]:
embeddings = []
for key in vocabulario.keys():
    if key not in glove:
        zeros = np.zeros(300)
        if key.isdigit():
            for i in range(0,300):
                zeros[i] = int(key)*0.000001
        embeddings.append({key: zeros})
    else:
        embeddings.append({key : glove[key][:300]})
                

In [26]:
p = np.zeros(300)

In [27]:
preguntas_embeddings = []

for i, preguntas in enumerate(preguntas_sin):
    palabras = tokenizer.tokenize(preguntas)
    words = []
    for j,palabra in enumerate(palabras):
        for h, word in enumerate(embeddings):
            for key in embeddings[h]:
                if(key == palabra):
                    words.append(embeddings[h][palabra][:300])
    if (j < 31):
        for j in range (j, 30):
            words.append(p)
    preguntas_embeddings.append(words)

In [28]:
# Calcular el tamano del vocabulario
vocabulario = []

for pregunta in preguntas_w:
    for palabra in pregunta:
        vocabulario.append(palabra)

In [29]:
vocabulario = set(vocabulario)

In [30]:
len(vocabulario)

68

In [31]:
# El tamano del vocabulario es necesario para poder crear la matriz de embeddings, con el numero de palabras
# totales que tengo
embedding_dim = 16
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

In [32]:
training_X = np.asarray(preguntas_embeddings[:800])
test_X = np.asarray(preguntas_embeddings[800:])
training_y = np.asarray(respuestas3[:800])
test_y = np.asarray(respuestas3[800:])


In [33]:
# Creamos un modelo donde el primer argumento de la capa embedding son las palabras totales que voy a procesar
# vectorizadas en un indice.
# El segundo argumento, es el tamano del vector embedding, que he fijado en 16.
# El tercer argumento, es el tamano o longitud maxima, que he definido para las preguntas. Numero total de palabras
# por pregunta.
model = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(36, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [34]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [35]:
history = model.fit(training_X, training_y, epochs = 300, validation_data=(test_X, test_y), batch_size = 128)

Epoch 1/300
7/7 [==============================] - 0s 41ms/step - loss: 1.3702 - accuracy: 0.3300 - val_loss: 1.3284 - val_accuracy: 0.3851
Epoch 2/300
7/7 [==============================] - 0s 15ms/step - loss: 1.3282 - accuracy: 0.3713 - val_loss: 1.3227 - val_accuracy: 0.3851
Epoch 3/300
7/7 [==============================] - 0s 13ms/step - loss: 1.2983 - accuracy: 0.3738 - val_loss: 1.3287 - val_accuracy: 0.3851
Epoch 4/300
7/7 [==============================] - 0s 11ms/step - loss: 1.2757 - accuracy: 0.3762 - val_loss: 1.3310 - val_accuracy: 0.3851
Epoch 5/300
7/7 [==============================] - 0s 13ms/step - loss: 1.2294 - accuracy: 0.4013 - val_loss: 1.3420 - val_accuracy: 0.3851
Epoch 6/300
7/7 [==============================] - 0s 16ms/step - loss: 1.1773 - accuracy: 0.4387 - val_loss: 1.3460 - val_accuracy: 0.3678
Epoch 7/300
7/7 [==============================] - 0s 13ms/step - loss: 1.1115 - accuracy: 0.5050 - val_loss: 1.3598 - val_accuracy: 0.3563
Epoch 8/300
7/7 [===

7/7 [==============================] - 0s 16ms/step - loss: 0.0747 - accuracy: 0.9675 - val_loss: 3.5409 - val_accuracy: 0.2529
Epoch 60/300
7/7 [==============================] - 0s 13ms/step - loss: 0.0820 - accuracy: 0.9638 - val_loss: 3.5684 - val_accuracy: 0.2586
Epoch 61/300
7/7 [==============================] - 0s 13ms/step - loss: 0.0846 - accuracy: 0.9625 - val_loss: 3.6241 - val_accuracy: 0.2644
Epoch 62/300
7/7 [==============================] - 0s 15ms/step - loss: 0.0725 - accuracy: 0.9725 - val_loss: 3.6897 - val_accuracy: 0.2816
Epoch 63/300
7/7 [==============================] - 0s 17ms/step - loss: 0.0777 - accuracy: 0.9650 - val_loss: 3.6534 - val_accuracy: 0.2529
Epoch 64/300
7/7 [==============================] - 0s 19ms/step - loss: 0.0876 - accuracy: 0.9625 - val_loss: 3.6252 - val_accuracy: 0.2471
Epoch 65/300
7/7 [==============================] - 0s 18ms/step - loss: 0.0738 - accuracy: 0.9688 - val_loss: 3.6752 - val_accuracy: 0.2586
Epoch 66/300
7/7 [========

7/7 [==============================] - 0s 20ms/step - loss: 0.0611 - accuracy: 0.9638 - val_loss: 4.2190 - val_accuracy: 0.2759
Epoch 118/300
7/7 [==============================] - 0s 18ms/step - loss: 0.0614 - accuracy: 0.9675 - val_loss: 4.2161 - val_accuracy: 0.2816
Epoch 119/300
7/7 [==============================] - 0s 19ms/step - loss: 0.0611 - accuracy: 0.9700 - val_loss: 4.2119 - val_accuracy: 0.2816
Epoch 120/300
7/7 [==============================] - 0s 16ms/step - loss: 0.0643 - accuracy: 0.9650 - val_loss: 4.2355 - val_accuracy: 0.2759
Epoch 121/300
7/7 [==============================] - 0s 14ms/step - loss: 0.0673 - accuracy: 0.9613 - val_loss: 4.2495 - val_accuracy: 0.2759
Epoch 122/300
7/7 [==============================] - 0s 13ms/step - loss: 0.0594 - accuracy: 0.9688 - val_loss: 4.2325 - val_accuracy: 0.2644
Epoch 123/300
7/7 [==============================] - 0s 17ms/step - loss: 0.0572 - accuracy: 0.9712 - val_loss: 4.2590 - val_accuracy: 0.2874
Epoch 124/300
7/7 [=

Epoch 175/300
7/7 [==============================] - 0s 15ms/step - loss: 0.0624 - accuracy: 0.9675 - val_loss: 4.6221 - val_accuracy: 0.2701
Epoch 176/300
7/7 [==============================] - 0s 12ms/step - loss: 0.0594 - accuracy: 0.9638 - val_loss: 4.6196 - val_accuracy: 0.2586
Epoch 177/300
7/7 [==============================] - 0s 17ms/step - loss: 0.0631 - accuracy: 0.9588 - val_loss: 4.6593 - val_accuracy: 0.2644
Epoch 178/300
7/7 [==============================] - 0s 18ms/step - loss: 0.0603 - accuracy: 0.9650 - val_loss: 4.6979 - val_accuracy: 0.2759
Epoch 179/300
7/7 [==============================] - 0s 19ms/step - loss: 0.0612 - accuracy: 0.9688 - val_loss: 4.7203 - val_accuracy: 0.2644
Epoch 180/300
7/7 [==============================] - 0s 18ms/step - loss: 0.0573 - accuracy: 0.9725 - val_loss: 4.7841 - val_accuracy: 0.2701
Epoch 181/300
7/7 [==============================] - 0s 16ms/step - loss: 0.0625 - accuracy: 0.9625 - val_loss: 4.8268 - val_accuracy: 0.2529
Epoch 

Epoch 233/300
7/7 [==============================] - 0s 13ms/step - loss: 0.0525 - accuracy: 0.9725 - val_loss: 5.1577 - val_accuracy: 0.2816
Epoch 234/300
7/7 [==============================] - 0s 11ms/step - loss: 0.0563 - accuracy: 0.9663 - val_loss: 5.1697 - val_accuracy: 0.2874
Epoch 235/300
7/7 [==============================] - 0s 12ms/step - loss: 0.0605 - accuracy: 0.9625 - val_loss: 5.1421 - val_accuracy: 0.2644
Epoch 236/300
7/7 [==============================] - 0s 15ms/step - loss: 0.0520 - accuracy: 0.9663 - val_loss: 5.1361 - val_accuracy: 0.2586
Epoch 237/300
7/7 [==============================] - 0s 13ms/step - loss: 0.0574 - accuracy: 0.9650 - val_loss: 5.1495 - val_accuracy: 0.2529
Epoch 238/300
7/7 [==============================] - 0s 13ms/step - loss: 0.0581 - accuracy: 0.9650 - val_loss: 5.1979 - val_accuracy: 0.2299
Epoch 239/300
7/7 [==============================] - 0s 16ms/step - loss: 0.0545 - accuracy: 0.9663 - val_loss: 5.2387 - val_accuracy: 0.2586
Epoch 

Epoch 291/300
7/7 [==============================] - 0s 15ms/step - loss: 0.0493 - accuracy: 0.9638 - val_loss: 5.6073 - val_accuracy: 0.2931
Epoch 292/300
7/7 [==============================] - 0s 19ms/step - loss: 0.0559 - accuracy: 0.9675 - val_loss: 5.6641 - val_accuracy: 0.2816
Epoch 293/300
7/7 [==============================] - 0s 19ms/step - loss: 0.0584 - accuracy: 0.9600 - val_loss: 5.6760 - val_accuracy: 0.2931
Epoch 294/300
7/7 [==============================] - 0s 21ms/step - loss: 0.0570 - accuracy: 0.9638 - val_loss: 5.7144 - val_accuracy: 0.2989
Epoch 295/300
7/7 [==============================] - 0s 28ms/step - loss: 0.0509 - accuracy: 0.9700 - val_loss: 5.7184 - val_accuracy: 0.3046
Epoch 296/300
7/7 [==============================] - 0s 18ms/step - loss: 0.0554 - accuracy: 0.9675 - val_loss: 5.6691 - val_accuracy: 0.3046
Epoch 297/300
7/7 [==============================] - 0s 16ms/step - loss: 0.0518 - accuracy: 0.9712 - val_loss: 5.6276 - val_accuracy: 0.2989
Epoch 

In [38]:
model2 = keras.Sequential([
    keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences= True, input_shape=training_X.shape)),
    keras.layers.LSTM (64, return_sequences = True),
    keras.layers.LSTM (128,  return_sequences = True),
    keras.layers.LSTM (256, return_sequences = True),
    keras.layers.LSTM (64),
    keras.layers.Dense(5, activation='softmax')
])
opt = keras.optimizers.RMSprop(
    learning_rate=0.001,
    rho=0.9,
    momentum=0.0,
    epsilon=1e-07,
    centered=False,
)

#opt = keras.optimizers.SGD(lr=0.00001, decay=1e-6, momentum=0.9, nesterov=True)
#opt = keras.optimizers.Adam(learning_rate = 0.0001)
model2.compile(loss='sparse_categorical_crossentropy',optimizer=opt, metrics=['accuracy'])

NameError: name 'vocab_size' is not defined

In [39]:
model2.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [40]:
history2 = model2.fit(training_X, training_y, epochs = 240, batch_size= 128, validation_data=(test_X, test_y))

Epoch 1/240
7/7 [==============================] - 8s 1s/step - loss: 1.6344 - accuracy: 0.3325 - val_loss: 1.5614 - val_accuracy: 0.3851
Epoch 2/240
7/7 [==============================] - 5s 730ms/step - loss: 1.4206 - accuracy: 0.3700 - val_loss: 1.3421 - val_accuracy: 0.3851
Epoch 3/240
7/7 [==============================] - 5s 724ms/step - loss: 1.3759 - accuracy: 0.3700 - val_loss: 1.3420 - val_accuracy: 0.2874
Epoch 4/240
7/7 [==============================] - 5s 763ms/step - loss: 1.3566 - accuracy: 0.3587 - val_loss: 1.4241 - val_accuracy: 0.3563
Epoch 5/240
7/7 [==============================] - 5s 740ms/step - loss: 1.3667 - accuracy: 0.3675 - val_loss: 1.4362 - val_accuracy: 0.3851
Epoch 6/240
7/7 [==============================] - 5s 736ms/step - loss: 1.3783 - accuracy: 0.3700 - val_loss: 1.3640 - val_accuracy: 0.3851
Epoch 7/240
7/7 [==============================] - 5s 725ms/step - loss: 1.3595 - accuracy: 0.3700 - val_loss: 1.3488 - val_accuracy: 0.3851
Epoch 8/240
7/7 

Epoch 59/240
7/7 [==============================] - 5s 750ms/step - loss: 1.1982 - accuracy: 0.4663 - val_loss: 1.5065 - val_accuracy: 0.3333
Epoch 60/240
7/7 [==============================] - 5s 763ms/step - loss: 1.1676 - accuracy: 0.4963 - val_loss: 1.5660 - val_accuracy: 0.1839
Epoch 61/240
7/7 [==============================] - 5s 785ms/step - loss: 1.1634 - accuracy: 0.5025 - val_loss: 1.6632 - val_accuracy: 0.1954
Epoch 62/240
7/7 [==============================] - 5s 736ms/step - loss: 1.2002 - accuracy: 0.4750 - val_loss: 1.6618 - val_accuracy: 0.2069
Epoch 63/240
7/7 [==============================] - 5s 769ms/step - loss: 1.0902 - accuracy: 0.5437 - val_loss: 1.5894 - val_accuracy: 0.2126
Epoch 64/240
7/7 [==============================] - 5s 757ms/step - loss: 1.1080 - accuracy: 0.5163 - val_loss: 1.5990 - val_accuracy: 0.2644
Epoch 65/240
7/7 [==============================] - 5s 775ms/step - loss: 1.1466 - accuracy: 0.5075 - val_loss: 1.5285 - val_accuracy: 0.2989
Epoch 

7/7 [==============================] - 6s 797ms/step - loss: 0.7112 - accuracy: 0.7412 - val_loss: 2.2391 - val_accuracy: 0.2816
Epoch 117/240
7/7 [==============================] - 6s 795ms/step - loss: 0.7743 - accuracy: 0.6913 - val_loss: 1.9540 - val_accuracy: 0.2816
Epoch 118/240
7/7 [==============================] - 5s 723ms/step - loss: 0.7407 - accuracy: 0.7125 - val_loss: 2.1045 - val_accuracy: 0.2356
Epoch 119/240
7/7 [==============================] - 6s 800ms/step - loss: 0.6677 - accuracy: 0.7362 - val_loss: 2.1611 - val_accuracy: 0.2874
Epoch 120/240
7/7 [==============================] - 5s 774ms/step - loss: 0.6845 - accuracy: 0.7525 - val_loss: 2.2713 - val_accuracy: 0.2471
Epoch 121/240
7/7 [==============================] - 6s 809ms/step - loss: 0.8031 - accuracy: 0.6812 - val_loss: 2.2671 - val_accuracy: 0.2759
Epoch 122/240
7/7 [==============================] - 6s 912ms/step - loss: 0.6718 - accuracy: 0.7437 - val_loss: 2.2528 - val_accuracy: 0.2644
Epoch 123/240

7/7 [==============================] - 6s 833ms/step - loss: 0.3281 - accuracy: 0.8800 - val_loss: 3.1701 - val_accuracy: 0.2816
Epoch 174/240
7/7 [==============================] - 5s 743ms/step - loss: 0.5928 - accuracy: 0.8037 - val_loss: 2.6822 - val_accuracy: 0.2644
Epoch 175/240
7/7 [==============================] - 6s 788ms/step - loss: 0.3649 - accuracy: 0.8575 - val_loss: 3.1208 - val_accuracy: 0.2586
Epoch 176/240
7/7 [==============================] - 6s 855ms/step - loss: 0.2850 - accuracy: 0.8900 - val_loss: 3.0008 - val_accuracy: 0.3103
Epoch 177/240
7/7 [==============================] - 5s 785ms/step - loss: 0.3724 - accuracy: 0.8612 - val_loss: 3.1367 - val_accuracy: 0.2874
Epoch 178/240
7/7 [==============================] - 5s 782ms/step - loss: 0.3193 - accuracy: 0.8800 - val_loss: 3.4754 - val_accuracy: 0.1724
Epoch 179/240
7/7 [==============================] - 5s 783ms/step - loss: 0.5733 - accuracy: 0.8037 - val_loss: 3.1535 - val_accuracy: 0.2644
Epoch 180/240

7/7 [==============================] - 6s 821ms/step - loss: 0.3315 - accuracy: 0.8838 - val_loss: 3.7289 - val_accuracy: 0.2701
Epoch 231/240
7/7 [==============================] - 6s 793ms/step - loss: 0.1373 - accuracy: 0.9350 - val_loss: 3.8879 - val_accuracy: 0.2644
Epoch 232/240
7/7 [==============================] - 6s 899ms/step - loss: 0.1883 - accuracy: 0.9187 - val_loss: 3.9005 - val_accuracy: 0.2586
Epoch 233/240
7/7 [==============================] - 6s 835ms/step - loss: 0.7068 - accuracy: 0.7800 - val_loss: 3.2241 - val_accuracy: 0.2701
Epoch 234/240
7/7 [==============================] - 7s 933ms/step - loss: 0.2465 - accuracy: 0.8900 - val_loss: 3.4799 - val_accuracy: 0.2701
Epoch 235/240
7/7 [==============================] - 7s 929ms/step - loss: 0.1461 - accuracy: 0.9375 - val_loss: 3.5630 - val_accuracy: 0.2759
Epoch 236/240
7/7 [==============================] - 7s 1s/step - loss: 0.1276 - accuracy: 0.9438 - val_loss: 3.8149 - val_accuracy: 0.2874
Epoch 237/240
7/